In [7]:
from fastai.text import * 
import html 
import pathlib

In [8]:
DATA_PATH=pathlib.Path('data/')
DATA_PATH.mkdir(exist_ok=True)
#! curl -O http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
#! tar -xzfv aclImdb_v1.tar.gz -C {DATA_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  35.4M      0  0:00:02  0:00:02 --:--:-- 35.4M
tar: Error opening archive: Failed to open 'v'


In [13]:
BOS = 'xbos' # beginning of sentence tag 
FLD = 'xfld' # data field tag 

PATH = pathlib.Path('data/aclImdb/')

In [14]:
CLAS_PATH=pathlib.Path('data/imdb_clas/')
CLAS_PATH.mkdir(exist_ok=True)

In [16]:
LM_PATH=pathlib.Path('data/imdb_lm/')
LM_PATH.mkdir(exist_ok=True)

In [22]:
import numpy as np
CLASSES = ['neg', 'pos', 'unsup']

def get_texts(path):
    '''
    input : text data with labels in front of sentence
    output : array text with associated label
    ''' 
    texts, labels = [], []
    for idx, label in enumerate(CLASSES):
        for fname in (path/label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
    return np.array(texts), np.array(labels)

trn_txts, trn_labels = get_texts(PATH/'train')
val_txts, val_labels = get_texts(PATH/'test')

In [23]:
len(trn_txts), len(val_txts)

(75000, 25000)

In [30]:
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_txts))
val_idx = np.random.permutation(len(val_txts))

75000

In [33]:
trn_texts = trn_txts[trn_idx]
val_texts = val_txts[val_idx]

trn_labels = trn_labels[trn_idx]
val_labels = trn_labels[val_idx]

In [37]:
import pandas as pd 
col_names = ['labels', 'text']
df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names) 

df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)

In [38]:
df_trn.head()

,labels,text
0,2,"I love all of Linda Howard's books, and was th..."
1,0,The Lack of content in this movie amazed me th...
2,1,You know what they say about the 70's..if you ...
3,2,"An old grandfather, Don Plutarco plays the vio..."
4,2,"At first, I actually had no idea that Billy Bl..."


In [39]:
df_val.head()

,labels,text
0,1,"I'd read all the negative reviews for ""Anna Ch..."
1,2,"Dude, I thought this movie rocked. Perfect for..."
2,2,A nurse travels to a rural psychiatric clinic ...
3,0,"An ""independant"" film that, from the back of t..."
4,2,This is just a very bad film. Miles looks as i...


In [40]:
(CLAS_PATH/'classes.txt').open('w').writelines(f'{o}\n' for o in CLASSES)
(CLAS_PATH/'classes.txt').open().readlines()

['neg\n', 'pos\n', 'unsup\n']

In [53]:
from sklearn.model_selection import train_test_split
trn_texts, val_texts = sklearn.model_selection.train_test_split(np.concatenate([trn_texts, val_texts]), test_size=0.1)
print(len(trn_texts), len(val_texts))

90000 10000


In [68]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)})
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)})

In [70]:
df_trn.to_csv(LM_PATH/'trn.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'val.csv', header=False, index=False)


In [73]:
re1 = re.compile(r'  +')
def fixup(x):
   x = x.replace('#39;', "'").replace('amp;', '&')
        .replace('#146;', "'").replace('nbsp;', ' ')
        .replace('#36;', '$').replace('\\n', "\n")
        .replace('quot;', "'").replace('<br />', "\n")
        .replace('\\"', '"').replace('<unk>','u_n')
        .replace(' @.@ ','.').replace(' @-@ ','-')
        .replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [74]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): 
        texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = texts.apply(fixup).values.astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

In [75]:
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [94]:
chunksize = 24000
df_trn = pd.read_csv(LM_PATH/'trn.csv', header=None, 
                     chunksize=chunksize)
df_val = pd.read_csv(LM_PATH/'val.csv', header=None, 
                     chunksize=chunksize)

In [84]:
tok_trn, trn_labels = get_all(df_trn, 1)
tok_val, val_labels = get_all(df_val, 1)

0


ValueError: invalid literal for int() with base 10: "Just to make it clear: yes, I think that this film tells an important story but am I the only one who had the feeling that this film is mostly about it's director? There are so many scenes where we j

In [96]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)})
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)})

In [105]:
df_trn.iloc[:, 2].values.astype(np.int64)

IndexError: single positional indexer is out-of-bounds